<a href="https://colab.research.google.com/github/jazkre/01ZLMA/blob/main/R/01ZLMA_zapoctova_uloha_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zadání zápočtové úlohy z předmětu 01ZLMA 2022/2023

https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients

Data Set Information:

This research aimed at the case of customers default payments in Taiwan and compares the predictive accuracy of probability of default among six data mining methods. From the perspective of risk management, the result of predictive accuracy of the estimated probability of default will be more valuable than the binary result of classification - credible or not credible clients





In [ ]:
# Install and load necessary libraries
#install.packages("readxl")
library(readxl)
library(tidyverse)

In [ ]:

# Original URL of the dataset
url <- "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"

# Temporary file to store the downloaded data
temp <- tempfile(fileext = ".xls")

# Download the file
download.file(url, temp, mode = "wb")

# Read the Excel file
df <- read_excel(temp,skip = 1,col_names = TRUE)

# Remove the temporary file
unlink(temp)

In [ ]:
df <- df %>% 
  rename(default = `default payment next month`)
df %>% head()
data <- df

In [ ]:
# Get column names that start with "bill_amt" or "pay_amt"
colnames(data) <- tolower(colnames(data))
bill_cols <- grep("^bill_amt", names(data), value = TRUE)
pay_cols <- grep("^pay_amt", names(data), value = TRUE)

# Compute min and max for each row from columns starting with "bill_amt" or "pay_amt"
data <- data %>%
  mutate(
    bill_min = pmap_dbl(select(., starts_with("bill_amt")), min, na.rm = TRUE),
    bill_max = pmap_dbl(select(., starts_with("bill_amt")), max, na.rm = TRUE),
    pay_min = pmap_dbl(select(., starts_with("pay_amt")), min, na.rm = TRUE),
    pay_max = pmap_dbl(select(., starts_with("pay_amt")), max, na.rm = TRUE)
  ) %>%
  select(-all_of(bill_cols), -all_of(pay_cols), -c("id","pay_2","pay_3","pay_4","pay_5"))

data %>% head()

In [ ]:
data <- data %>%
  filter(!(bill_min <= 0 | pay_max <= 0),
         !(marriage == 0 | marriage == 3),
         !(education == 0)) %>%
  mutate(education = ifelse(education >= 3, 3, education)) %>%
  mutate(sex = factor(sex),
        education = factor(education),
        marriage = factor(marriage))

In [ ]:
summary(data)
dim(data)

In [ ]:
data_table <- data %>%
   dplyr::select(default,age, sex, education, marriage, limit_bal) %>%
   mutate(age             = cut(age, breaks=c(-Inf, 25,30,35,40,Inf),labels=c("20","25","30","35","40")),
          limit_bal  = cut(limit_bal, breaks=c(-Inf,50000, 100000,150000,200000,Inf),labels=c("50","100","150","200","250"))) %>%
  group_by(age,limit_bal,sex) %>%
  summarise(n = n(),
         default_yes = sum(default),
         default_no = n - sum(default)
        ) %>%
  select(-c("n"))       

data_table %>% head()

In [ ]:
install.packages("caret")
library(caret)

# Set the seed for reproducibility
set.seed(42)

# Create a stratified random sample of the row numbers
train_rows <- createDataPartition(data$default, p = 0.7, list = FALSE)

# Split the data into training and testing sets
train_data <- data[train_rows, ]
test_data <- data[-train_rows, ]

In [ ]:
dim(train_data)
dim(test_data)

## Zadání

## 01 - Grafické zobrazení dat a jejich kontrola

Použijte dataset `data_train` a `data_test`. 

* Navrhněte vhodný způsob zobrazení dat, který vám pomůže určit, zdali použití logistické regrese pro predikci nesplacení úvěru je možné a vhodné při použítí zvolených trénovacích a testvacích dat.
* Vykreslete boxploty pro různé numerické proměnné, odezvu `default` a barevn2 odlište kategorické proměnné. Je z těchto obrázků možné odhadnout jaká proměnná bude mít vliv na odezvu? (jen `data_train`).
* Pro vybrané skupiny z předešlé otázky navrhněte vhodný statistický test, proveďte ho a řekněte zdali je vámi zvolená proměnná statisticky významná pro určení schopnosti splácet (jen `data_train`). 

## 02 - Logistická regrese agregovaná tabulková data

použijte `data_table`





* Vytvořte odezvu pro logistický model binomického rozdělení a sestrojte nulový model. Jaká je ve zkoumané populaci  průměrná šance na neschopnost splácet úvěr  a jaká je pravděpodobnost nesplacení úvěru? 

* Sestrojte model, kde závisí výskyt nesplacení úvěru pouze na proměnné `limit_bal`. Je vliv limitu významná proměnná na hladině významnosti 0.05? Pokud ano, zjistěte, kolikrát mají lidé s limitem větším než 100 vetší šanci na neschopnost splácet než lidé s limitem nad 200.

* Sestrojte model, kde závisí výskyt neschopnosti splácet pouze na věku `age`.  Je věk  statisticky významná proměnná na hladině významnosti 0.01? Pokud ano, zjistěte, kolikrát mají pacienti ve věku mezi 35-40 menší šanci na  neschopnost splácet než lidé ve věku 20-25.

* Předpokládejte, že šance na neschopnost splácet roste exponenciálně s hodnotou limitu a klesá exponenciálně s věkem. Vytvořte příslušné numerické spojité prediktory. Sestrojte model, kde šance na neschopnost splácet bude záležet na numerických hodnotách tlaku a věku (bez interakce). Jaký je poměr šancí na neschopnost splácet mezi lidmi lišícími se limitem o 50000 a majícími stejný věk? 

* Otestujte předešlý model proti saturovanému modelu. Má tento test opodstatnění? Okomentujte jeho výsledek. 

## 03 - Poissonovská regrese na agregovaná tabulková data (4/7)

použijte  `data_table`


* Upravte tabulku do potřebného formátu a sestrojte čistě aditivní loglineární model pro četnosti pozorování ve skupinách, který předpokládá vzájemnou
nezávislost mezi všemi skupinovými prediktory.

* Na základě předchozího modelu, jaký je odhad šance na neschopnost splácet mezi všemi vybranými lidmi ve skupině a jaký je odhad pravděpodobnosti neschopnosti splácet?

* Sestavte model, který obsahuje všechny možné dvojné interakce mezi klasifikačními veličinami a porovnejte tento model s předchozím modelem bez interakcí. Je tento model statisticky významně lepší?

* Sestavte saturovaný model. Je tento model statisticky významně lepší než předchozí s dvojtými interakcemi? Vyberte model který je podle vás nejlepší.

* Na základě vybraného modelu, je ve všech věkových skupinách stejná souvislost mezi limitem a šancí splácet?

* Na základě vybraného modelu, je pro obě pohlaví stejná souvislost mezi limitem a šancí splácet?

* Ve skupině co se týče vzdělání je nějvětší souvislost mezi schopností splácet a limitem?


## 04 - Logistická regrese - statistika (5/8)

Nyní již používejte `data_train`

* Vypište kontingenční tabulku pro proměnné pohlaví (`sex`) a schopnost splácet  (`default`). Ručně z tabulky spočtěte empirický poměr šancí na schopnost splácet (muž vs. žena) a pravděpodobnost onemocnění pro ženu a muže. Výsledek porovnejte s výsledky z logistické regrese, kde jediným prediktorem bude právě proměnná `sex` a odezvou proměnná `default`. Pro poměr šancí uveďte i 95% konfidenční interval a okometujte, zdali mají ženy signifikantně menší šanci na schopnost splácet.

* Vypište kontingenční tabulku pro proměnné vzdělání (`education`) a schopnost splácet (`default`). Ručně z tabulky spočtěte empirický poměr šancí na onemocnění srdce (type 0: asymptomatic, proti ostatním) a pravděpodobnosti onemocnění pro jednotlivé typy. Výsledek porovnejte s výsledky z logistické regrese, kde jediným prediktorem bude právě proměnná `education` a odezvou proměnná `default`. Pro poměr šancí uveďte i 95% konfidenční interval a okometujte, zdali mají s lepším vzděláním mají menší šanci na neschopnost splácet úvěr.

* Sestavte model, kde použijete všechny dostupné proměnné (jak faktorové tak numerické). Pomocí deviančních testů model postupně redukujte. Výsledný model porovnejte s modelem, který byste obdrželi při použití automatického výběru s funkcí `step()`.

* Pro vybraný model spočtěte šanci na schopnost splácet pro muže oproti ženám, včetně 95% konfidenčních intervalů. Stejně tak pro manželství a vzdělání. Jak se výsledek změnil oproti jednoduchým modelům a jak byste tuto změnu vysvětlili?

* Použijte váš model a spočtěte odhad pravděpodobnosti schopnosti splácet pro testovací data vykreslete pro proměnou `limit_bal` predikční konfidenční intervaly. 
Nápověda krom cvičení, také třeba zde: https://fromthebottomoftheheap.net/2017/05/01/glm-prediction-intervals-i/

* Na základě trénovacích dat zvolte vhodný treshold pro určení splatí / nesplatí a na testovacích datech spočtěte Accuracy a vykreslete ROC křivku spolu s AUC.





## 05 - Logistická regrese -  strojové učení

* Sestavte workflow (pipeline) na trenovacích datech pro logistickou regresi s použitím regularizace (elestic-net), která bude obsahovat 
 *   Přípravu proměnných, transformaci, one-hot encoding, normalizaci, ... (Možno použít recipe z tidy models)
 *   Hledání "optimálního" nastavení hyperparametru v regularizaci. 
 *   k-fold cross validaci

* Pomocí předchozí pipline/workflow vyberte hodnotu hyperparametru. Pokud chceme uvedený algoritmus použít k detekci pacientů s nemocí srdce, jakou statistiku bychom měli sledovat abychom omylem nepropustili nemocného domů jako zdravého? 

* Spočtěte a porovnejte statistiky používané při binární klasifikaci na trénovacích a testovacích datech. Vykreslete ROC křivku a spočtěte plochu pod ní jak pro testovací tak trénovací data. Co můžeme říci o kvalitě modelu z této sekce 05 a modelu z předchozí 04?




## Poznámky k odevzdání a vypracování

* Úkol vypracujte ve skupině. Pokud se budete s někým radit mimo skupinu, nezapomeňte u dané odpovědi uvést zdroj vašich informací. 

* Úkol můžete odevzdávat jako spustitelný jupyter notebook na google colab (stačí odevzdat link na github). Před odevzdáním ZKONTROLUJTE, že buňky na sebe navazují a že lze NB postupně projít a jednotlivé buňky nezahlásí chybu.

* Deadline pro odevzdání je stanoven na sředu 21.6.2022.

* Není možno odevzdání opakovat. Odevzdaný úkol zkontroluji a buď zápočet získáte, nebo ne. Je vyžadováno mít alespoň 75% úkolů správně.

* V případě otázek se ptejte veřejně v general kanále v MS Teams. Ostatní mohou mít podobné problémy a tudíž pokud je někde například nejasné zadání, tak ať mají všichni stejnou příležitost.

* Možno psát a odevzdat v Pythonu.